Let's go the starting point...

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request

contents = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read()
soup = BeautifulSoup(contents, "lxml")

In [ ]:
table = soup.find('table')
tb = table.find('tbody')
linhas = tb.find_all('tr')
lista =[]
for linha in linhas:  
    elementos = linha.find_all('td')
    listinha =[]
    for elemento in elementos:
        listinha.append(elemento.string)
    lista.append(listinha)

df = pd.DataFrame(lista, columns=['Postcode','Borough','Neighbourhood'])

df =df.dropna()

df['Neighbourhood']= df['Neighbourhood'].str.rstrip(']\n')
df['Borough']=df['Borough'].str.rstrip()
df = df[(df != "Not assigned").all(axis =1)]
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join)
df = pd.DataFrame({"Postcode":df.index.get_level_values("Postcode"), "Borough":df.index.get_level_values("Borough"), "Neighbourhood":df.values})
df.head(10)

All right, now we have the data frame again, so let's set the coordinates.
But first, we need the package 


In [ ]:
!pip install geocoder

In [ ]:
import geocoder

getting every lat/long:

In [ ]:
latitude=[]
longitude=[]
for postal_code in df['Neighbourhood']:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

In [92]:
df['Latitude'] = latitude
df['Longitude'] = longitude
df.head(10)

,Postcode,Borough,Neighbourhood,latitude,Latitude,Longitude
0,M1E,Scarborough,Guildwood,None,43.752743,-79.192777
1,M1K,Scarborough,East Birchmount Park,None,43.711087,-79.269800
2,M1M,Scarborough,Scarborough Village West,None,43.776426,-79.231752
3,M1N,Scarborough,Cliffside West,None,43.702314,-79.250164
4,M1T,Scarborough,"Clarks Corners, Sullivan",None,43.647361,-79.381123
5,M1V,Scarborough,"L'Amoreaux East, Steeles East",None,43.819756,-79.324284
6,M1W,Scarborough,L'Amoreaux West,None,43.799586,-79.310051
7,M2J,North York,"Fairview, Oriole",None,43.765502,-79.364245
8,M2N,North York,Willowdale South,None,43.780584,-79.406659
9,M2P,North York,York Mills West,None,43.752948,-79.362137


Seems right!